In [ ]:
!pip3 install face_recognition

In [ ]:
# import required libraries
import glob
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
import json
import copy
import random
import torch
import torchvision
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import models
import torch
from torch.autograd import Variable
import time
import sys
import seaborn as sn
from sklearn.metrics import confusion_matrix
import torch.optim as optim

In [ ]:
# load videos to memory

video_files =  glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_dfdcfake/*.mp4')
video_files += glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_FFReal/*.mp4')
video_files += glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_FFFake/*.mp4')
video_files += glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_dfdcreal/*.mp4')
video_files += glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_CelebReal/*.mp4')
video_files += glob.glob('/content/drive/MyDrive/Colab Notebooks/Preprocess_CelebFake/*.mp4')

random.shuffle(video_files)
random.shuffle(video_files)
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<100):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))


In [ ]:
# create custom dataset class

class video_dataset(Dataset):
    def __init__(self,video_names,labels,sequence_length,transform = None):
        self.video_names = video_names
        self.labels = labels
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        temp_video = video_path.split('/')[-1]
        #print(temp_video)
        label = self.labels.iloc[(labels.loc[labels["File"] == temp_video].index.values[0]),1]
        if(label == 'FAKE'):
          label = 0
        if(label == 'REAL'):
          label = 1
        for i,frame in enumerate(self.frame_extract(video_path)):
          frames.append(self.transform(frame))
          if(len(frames) == self.count):
            break
        frames = torch.stack(frames)
        frames = frames[:self.count]
        
        return frames,label

    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path) 
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image

In [ ]:
#count the number of fake and real videos
def number_of_real_and_fake_videos(data_list):
  header_list = ["File","Label"]
  lab = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_labels.csv',names=header_list)
  fake = 0
  real = 0
  for i in data_list:
    temp_video = i.split('/')[-1]
    
    label = lab.iloc[(labels.loc[labels["File"] == temp_video].index.values[0]),1]
    if(label == 'FAKE'):
      fake+=1
    if(label == 'REAL'):
      real+=1
  return real,fake

In [ ]:
# load the labels and video in data loader
header_list = ["File","Label"]
labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/all_labels.csv',names=header_list)

train_videos = video_files[:int(0.8*len(video_files))]
temp = video_files[int(0.8*len(video_files)):]
valid_videos = temp[:int(0.5*len(temp))]
test_videos = temp[int(0.5*len(temp)):]

print("train : " , len(train_videos))
print("valid : " , len(valid_videos))
print("test : ",len(test_videos))

real,fake = number_of_real_and_fake_videos(train_videos)
print("TRAIN: ", "Real:",real," Fake:",fake)
real,fake = number_of_real_and_fake_videos(valid_videos)
print("VALID: ", "Real:",real," Fake:",fake)
real,fake = number_of_real_and_fake_videos(test_videos)
print("TEST: ", "Real:",real," Fake:",fake)

In [ ]:
im_size = 240
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])


train_data = video_dataset(train_videos,labels,sequence_length = 1,transform = transforms)
#print(train_data)

val_data = video_dataset(valid_videos,labels,sequence_length = 1,transform = transforms)
test_data = video_dataset(test_videos,labels,sequence_length = 1,transform = transforms)

train_loader = DataLoader(train_data,batch_size = 4,shuffle = True,num_workers = 4)
valid_loader = DataLoader(val_data,batch_size = 4,shuffle = True,num_workers = 4)
test_loader = DataLoader(test_data,batch_size = 4,shuffle = True,num_workers = 4)

In [ ]:
!pip install efficientnet-pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
#importing the pretrained EfficientNet model

model = EfficientNet.from_pretrained('efficientnet-b1')

# Freeze weights
for param in model.parameters():
    param.requires_grad = False
in_features = model._fc.in_features


# Defining Dense top layers after the convolutional layers
model._fc = nn.Sequential(
    nn.BatchNorm1d(num_features=in_features),    # normalise
    nn.Linear(in_features, 512),                 # linear transformation
    nn.ReLU(),                                   # activation function
    nn.BatchNorm1d(512),                         # normalise
    nn.Dropout(0.4),                             # During training, randomly zeroes some of the 
                                                 # elements of the input tensor with probability p 
                                                 # using samples from a Bernoulli distribution.
    nn.Linear(512, 2),                           # linear transformation
    
    )

print(model._fc)

use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()


In [ ]:
dummy_input = torch.randn(10, 3, 380, 380).type(torch.cuda.FloatTensor)
out = model(dummy_input)
print(out)

In [ ]:
# selecting loss function
criterion_transfer = nn.CrossEntropyLoss()

#using Adam classifier
optimizer_transfer = optim.Adam(model.parameters(), lr=0.00001, weight_decay = 0.00001)

n_epochs = 20


In [ ]:
# Creating the function for training
def train(n_epochs, train_loader,valid_loader, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    trainingloss = []
    validationloss = []

    for epoch in range(1, n_epochs+1):
        # initialize the variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # training the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # move to GPU
            #print(data.shape)
            batch,seq,c,h,w = data.shape
            data = data.view(batch*seq, c, h, w)
            if use_cuda:
                data, target = data.cuda(), target.type(torch.cuda.LongTensor)
          
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
           
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
    
        ######################    
        # validating the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(valid_loader):
            batch,seq,c,h,w = data.shape
            data = data.view(batch*seq, c, h, w)
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        train_loss = train_loss/len(train_videos)
        valid_loss = valid_loss/len(valid_videos)

        trainingloss.append(train_loss)
        validationloss.append(valid_loss)

        # printing training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
       
        ## saving the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            
            valid_loss_min = valid_loss
            
    # return trained model
    return model, trainingloss, validationloss

In [ ]:
model_transfer, train_loss, valid_loss = train(n_epochs, train_loader,valid_loader, model, optimizer_transfer, criterion_transfer, use_cuda, 'efficientnet-bx-2fc.pt')

In [ ]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('efficientnet-bx-2fc.pt'))

In [ ]:
# Defining the test function

def test(test_loader, model, criterion, use_cuda):

    # monitoring test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    preds = []
    targets = []

    model.eval()
    for batch_idx, (data, target) in enumerate(test_loader):
        # moving to GPU
        batch,seq,c,h,w = data.shape
        data = data.view(batch*seq, c, h, w)
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # updating average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # converting the output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        preds.append(pred)
        targets.append(target)
        # compare predictions
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
    
    return preds, targets

# calling test function
preds, targets = test(test_loader, model_transfer, criterion_transfer, use_cuda)


In [ ]:
#converting the tensor object to a list for metric functions

preds2, targets2 = [],[]

for i in preds:
  for j in range(len(i)):
    preds2.append(i.cpu().numpy()[j])
for i in targets:
  for j in range(len(i)):
    targets2.append(i.cpu().numpy()[j])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
#Computing the accuracy
acc = accuracy_score(targets2, preds2)
print("Accuracy: ", acc)

In [ ]:
cm = confusion_matrix(targets2, preds2)

#Create an object for the confusion matrix display class
cmob = ConfusionMatrixDisplay(cm, display_labels=['Real', 'Fake'])


# The plot() function has to be called for the sklearn visualization
cmob.plot()

# Use the Axes attribute 'ax_' to get to the underlying Axes object.
cmob.ax_.set(title='Confusion Matrix with labels', 
            xlabel='Predicted', 
            ylabel='Actual')

# Finally, call the matplotlib show() function to display the visualization
plt.show()